In [6]:
import pandas as pd     #(version 1.0.0)
import plotly           #(version 4.5.0)
import plotly.express as px

import dash             #(version 1.8.0)
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import jupyter_dash

app = jupyter_dash.JupyterDash(__name__)

In [9]:
# read gzip csv 
# (see https://stackoverflow.com/questions/39263929/how-can-i-read-tar-gz-file-using-pandas-read-csv-with-gzip-compression-option)
df_orig = pd.read_csv("DOHMH_New_York_City_Restaurant_Inspection_Results.csv.gz", compression='gzip')

In [10]:
df_orig.head(5)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,41371617,GU,Manhattan,155,EAST 52 STREET,10022.0,2127521142,Japanese,06/20/2018,Violations were cited in the following area(s).,...,01/03/2020,Cycle Inspection / Initial Inspection,40.757511,-73.970881,106.0,4.0,10000.0,1036468.0,1.013070e+09,MN19
1,41539256,WINGSTOP,Queens,3114,STEINWAY STREET,11103.0,7187779464,Chicken,11/09/2016,Violations were cited in the following area(s).,...,01/03/2020,Cycle Inspection / Initial Inspection,40.760732,-73.917578,401.0,22.0,15500.0,4010524.0,4.006580e+09,QN70
2,41637941,KENNEDY CHICKEN &SANDWICHES,Bronx,101,EAST 167 STREET,10452.0,3472716218,Chicken,05/14/2019,Violations were cited in the following area(s).,...,01/03/2020,Cycle Inspection / Initial Inspection,40.835050,-73.919293,204.0,16.0,19700.0,2002850.0,2.024650e+09,BX63
3,41689402,KARE THAI,Manhattan,752,10 AVENUE,10019.0,2127657800,Thai,12/08/2016,Violations were cited in the following area(s).,...,01/03/2020,Cycle Inspection / Initial Inspection,40.765455,-73.991210,104.0,3.0,13300.0,1026710.0,1.010610e+09,MN15
4,50006618,SABOR ECUATORIANO BAKERY,Queens,4042,82ND ST,11373.0,9178327161,Bakery,07/13/2017,Violations were cited in the following area(s).,...,01/03/2020,Cycle Inspection / Initial Inspection,40.746465,-73.883737,404.0,25.0,26901.0,4036881.0,4.014920e+09,QN29


In [11]:
df_orig.shape, df_orig.columns

((398593, 26),
 Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
        'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
        'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
        'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE', 'Latitude', 'Longitude',
        'Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL',
        'NTA'],
       dtype='object'))

In [12]:
df = df_orig.copy()
df['INSPECTION DATE'] = pd.to_datetime(df['INSPECTION DATE'])
df = df.groupby(['INSPECTION DATE','CUISINE DESCRIPTION','CAMIS'], as_index=False)['SCORE'].mean()
df = df.set_index('INSPECTION DATE')
df = df.loc['2016-01-01':'2019-12-31']
df = df.groupby([pd.Grouper(freq="M"),'CUISINE DESCRIPTION'])['SCORE'].mean().reset_index()
df[:5]

,INSPECTION DATE,CUISINE DESCRIPTION,SCORE
0,2016-01-31,American,12.285714
1,2016-01-31,Asian,18.000000
2,2016-01-31,Bagels/Pretzels,11.000000
3,2016-01-31,Bakery,8.000000
4,2016-01-31,Café/Coffee/Tea,6.200000


In [13]:
df["CUISINE DESCRIPTION"].unique()

array(['American', 'Asian', 'Bagels/Pretzels', 'Bakery',
       'Café/Coffee/Tea', 'Caribbean', 'Chinese', 'Chinese/Japanese',
       'Continental', 'Delicatessen', 'Ethiopian', 'French', 'Hamburgers',
       'Indian', 'Italian', 'Japanese', 'Jewish/Kosher', 'Korean',
       'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
       'Mexican', 'Middle Eastern', 'Peruvian', 'Pizza', 'Pizza/Italian',
       'Russian', 'Sandwiches/Salads/Mixed Buffet', 'Spanish', 'Tapas',
       'Thai', 'Armenian', 'Barbecue', 'Brazilian', 'Chicken', 'Creole',
       'Donuts', 'Eastern European', 'Greek',
       'Ice Cream, Gelato, Yogurt, Ices', 'Irish',
       'Juice, Smoothies, Fruit Salads', 'Mediterranean', 'Moroccan',
       'Pancakes/Waffles', 'Sandwiches', 'Turkish',
       'Vietnamese/Cambodian/Malaysia', 'Afghan', 'Bangladeshi', 'Cajun',
       'Egyptian', 'Hawaiian', 'Nuts/Confectionary', 'Other', 'Salads',
       'Seafood', 'Soul Food', 'Soups & Sandwiches', 'Steak', 'Tex-Mex',

In [14]:
#---------------------------------------------------------------
app.layout = html.Div([

    html.Div([
        dcc.Graph(id='our_graph')
    ],className='nine columns'),

    html.Div([

        html.Br(),
        html.Label(['Choose 3 Cuisines to Compare:'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='cuisine_one',
            options=[{'label':x, 'value':x} for x in df.sort_values('CUISINE DESCRIPTION')['CUISINE DESCRIPTION'].unique()],
            value='African',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose Cuisine...',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'),

        dcc.Dropdown(id='cuisine_two',
            options=[{'label':x, 'value':x} for x in df.sort_values('CUISINE DESCRIPTION')['CUISINE DESCRIPTION'].unique()],
            value='Asian',
            multi=False,
            clearable=False,
            persistence='string',
            persistence_type='session'),

        dcc.Dropdown(id='cuisine_three',
            options=[{'label':x, 'value':x} for x in df.sort_values('CUISINE DESCRIPTION')['CUISINE DESCRIPTION'].unique()],
            value='Donuts',
            multi=False,
            clearable=False,
            persistence='string',
            persistence_type='local'),

    ],className='three columns'),

])

#---------------------------------------------------------------

@app.callback(
    Output('our_graph','figure'),
    [Input('cuisine_one','value'),
     Input('cuisine_two','value'),
     Input('cuisine_three','value')]
)

def build_graph(first_cuisine, second_cuisine, third_cuisine):
    dff=df[(df['CUISINE DESCRIPTION']==first_cuisine)|
           (df['CUISINE DESCRIPTION']==second_cuisine)|
           (df['CUISINE DESCRIPTION']==third_cuisine)]
    # print(dff[:5])

    fig = px.line(dff, x="INSPECTION DATE", y="SCORE", color='CUISINE DESCRIPTION', height=600)
    fig.update_layout(yaxis={'title':'NEGATIVE POINT'},
                      title={'text':'Restaurant Inspections in NYC',
                      'font':{'size':28},'x':0.5,'xanchor':'center'})
    return fig

#---------------------------------------------------------------

if __name__ == '__main__':
    app.run_server(debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Nov/2020 15:27:10] "GET /_alive_75a142a0-b7c3-45cb-a340-5cae1543e716 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


127.0.0.1 - - [28/Nov/2020 15:27:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2020 15:27:13] "GET /assets/bWLwgP.css?m=1606593073.771366 HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2020 15:27:13] "GET /_dash-component-suites/dash_renderer/react@16.v1_8_3m1606129149.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2020 15:27:13] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_8_3m1606129149.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2020 15:27:13] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_8_3m1606129149.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2020 15:27:13] "GET /_dash-component-suites/dash_renderer/dash_renderer.v1_8_3m1606129149.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2020 15:27:13] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2020 15:27:13] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2020 15:27:14] "GET /_dash-component-suites/dash_core_components/async-dropdown.v1_13_0m1604001894.js HTTP/1.1" 200 -
127.0